In [1]:
import scipy.stats as stats
import pandas as pd
import psycopg2
import numpy as np
import math
import json
import os
import backtrader as bt
import datetime
import matplotlib

In [2]:
# Connect to db
conn = psycopg2.connect('dbname=securities user=postgres')
cur = conn.cursor()

In [3]:
# Get all price data
query_all = '''
    SELECT *
    FROM prices
    ORDER BY symbol ASC, datetime ASC
'''
cur.execute(query_all)
data = cur.fetchall()
price_df = pd.DataFrame(data, columns = ['symbol', 'date', 'open', 'high', 'low', 'close'])
price_df.loc[:, 'date'] = pd.to_datetime(price_df['date'])
price_df = price_df.set_index('date')

# Get list of all unique tickers
ticker_list = price_df.symbol.unique()

In [4]:
class SMAIndicator(bt.Indicator):
    lines = ('sma',)
    params = {
        'long_ma': None,
        'short_ma': None,
    }
    
    def __init__(self):
        self.addminperiod(max(self.params.long_ma, self.params.short_ma))
        self.long_period = max(self.params.long_ma, self.params.short_ma)
        self.short_period = min(self.params.long_ma, self.params.short_ma)
    
    def next(self):
        sma_long = np.mean(self.data.get(size = self.long_period))
        sma_short = np.mean(self.data.get(size = self.short_period))
        self.lines.sma[0] = sma_short - sma_long

In [36]:
class TestStrategy(bt.Strategy):
    
    params = {
        'trade_freq': None,
        'feed_name': None,
    }
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.sma = SMAIndicator(long_ma = 30, short_ma = 10)
        self.days = 0
        
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            self.inds[d]['sma'] = self.sma
            
        print(len(self.inds))
        
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, PRICE: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(
            'OPERATION PROFIT, GROSS %.2f, NET %.2f' % (
                trade.pnl,
                trade.pnlcomm,
            )
        )
        
    def next(self):
        self.days += 1
        
        if self.order:
            return
        
        if self.days > self.params.trade_freq - 1:
            self.days = 0
            for i, d in enumerate(self.datas):
                dt, dn = self.datetime.date(), d._name
                
                if self.inds[d]['sma'][0] > 0:
                    self.log('BUY CREATE, %s, %.2f' % (dn, d.close[0]))
                    self.order = self.buy(data = d, size = 2)
                else:
                    self.log('SELL CREATE, %s, %.2f' % (dn, d.close[0]))
                    self.order = self.sell(data = d, size = 2)

In [37]:
cerebro = bt.Cerebro()

# Load all feeds into Cerebro
for ticker in ['SPY', 'IWO']:
    ticker_df = price_df[price_df['symbol'] == ticker]
    cerebro.adddata(bt.feeds.PandasData(dataname = ticker_df, name = ticker, datetime = -1))

cerebro.addstrategy(TestStrategy, trade_freq = 10)

0

In [38]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2
2019-10-22, BUY CREATE, SPY, 299.01
2019-10-22, BUY CREATE, IWO, 195.80
2019-10-23, BUY EXECUTED, PRICE: 298.73, Cost: 597.46, Comm 0.00
2019-10-23, BUY EXECUTED, PRICE: 195.81, Cost: 391.62, Comm 0.00
2019-11-05, BUY CREATE, SPY, 307.03
2019-11-05, BUY CREATE, IWO, 202.61
2019-11-06, BUY EXECUTED, PRICE: 307.03, Cost: 614.06, Comm 0.00
2019-11-06, BUY EXECUTED, PRICE: 202.41, Cost: 404.82, Comm 0.00
2019-11-19, BUY CREATE, SPY, 311.93
2019-11-19, BUY CREATE, IWO, 205.01
2019-11-20, BUY EXECUTED, PRICE: 311.28, Cost: 622.56, Comm 0.00
2019-11-20, BUY EXECUTED, PRICE: 204.24, Cost: 408.48, Comm 0.00
2019-12-04, BUY CREATE, SPY, 311.46
2019-12-04, BUY CREATE, IWO, 209.09
2019-12-05, BUY EXECUTED, PRICE: 312.23, Cost: 624.46, Comm 0.00
2019-12-05, BUY EXECUTED, PRICE: 209.84, Cost: 419.68, Comm 0.00
2019-12-18, BUY CREATE, SPY, 319.59
2019-12-18, BUY CREATE, IWO, 212.84
2019-12-19, BUY EXECUTED, PRICE: 319.80, Cost: 639.60, Comm 0.00
2019-12-19, BUY EX